In [63]:
import time 
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import InvalidArgumentException
from torrequest import TorRequest
from stem import Signal
from stem.control import Controller
from stem import Signal
from stem.control import Controller
from random_useragent.random_useragent import Randomize
import requests as re
import bs4
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import datetime as dt

In [64]:
def crawler_speed(min,max):
    random=randint(min,max)
    sleep(random)

In [65]:
def user_a():
    m=randint(0,6)
    agents=[]
    # Get aspect ratio list.
    r_agent = Randomize()
    r_agent.get_aspect_ratio_list() # returns ['3:2', '4:3', '5:3', '5:4', '16:9', '16:10'].
  

    
    # Takes 2 arguments (self, aspect_ratio).
    r_agent.random_resolution('3:2') # returns screen resolution.
    
    # Takes 3 arguments (self, device_type, os)
    agents.append(r_agent.random_agent('desktop','linux')) # returns 'Desktop / Linux'
    agents.append(r_agent.random_agent('desktop','mac')) # returns 'Desktop / Linux'
    agents.append(r_agent.random_agent('desktop','windows')) # returns 'Desktop / Macintosh'

    agents.append(r_agent.random_agent('tablet','android')) # returns 'Tablet / Android'
    agents.append(r_agent.random_agent('tablet','ios')) # returns 'Tablet / iOS'

    agents.append(r_agent.random_agent('smartphone','android')) # returns 'Smartphone / Android'
    agents.append(r_agent.random_agent('tablet','ios')) # returns 'Smartphone / iOS'
    return(agents[m])
def get_tor_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
    session.proxies = {'http':  'socks5://127.0.0.1:9050',
                       'https': 'socks5://127.0.0.1:9050'}
    return session
def _set_new_ip():
    with Controller.from_port(port=9051) as controller:
        controller.authenticate(password='rastafari')
        controller.signal(Signal.NEWNYM)

class ProxyMiddleware(object):
    def process_request(self, request, spider):
        _set_new_ip()
        request.meta['proxy'] = 'http://127.0.0.1:8118'
        spider.log('Proxy : %s' % request.meta['proxy'])

In [66]:
def craikilla(minp,maxp,min,max,srch):
    #import as database for make and model
    k=pd.read_pickle("cars.pickle")
    search=srch
    string_len=len(search)
    search=search.split()
    string_len=len(search)

    x4=str()
    for st in range(string_len-1):
        x4=x4+search[st]+"+"
    x='https://phoenix.craigslist.org/search/sss?query='
    x3='&sort=rel&min_price='+str(minp)+'&max_price='+str(maxp)
    x4=x4+search[len(search)-1]
    link1=x+x4+x3
    l1=link1
    #Here i define my random user for this scrap
    hed={'User-Agent':user_a()}
    #The ip is reset before the request is called
    _set_new_ip()  
    #untraceable tor session is requested
    session=get_tor_session()
    res=session.get(link1,headers=hed)
    #Beautiful soup object is created
    soup=bs4.BeautifulSoup(res.text,'html')
    
    total_results=soup.find('span',"totalcount")

    num_of_results=total_results.text
    num_of_results=num_of_results.replace("results"," ")
    num_of_results=num_of_results.replace(",","")
    resultados=int(num_of_results)
    print(resultados)        
    print(link1)
    #These are all the post
    allelements_on_page=[]
    titles=[]
    tile_links=[]
    prices=[]
    hood=[]
    model=[]
    make=[]
    year=[]
    date=[]
    
    
    
    #age = input("Do you want to continue?")
    b=1
    #if age =='yes':
    pags=round(resultados/120)
    print(pags)
    for x in range(pags):
        try:
            crawler_speed(min,max)
            res=session.get(link1,headers=hed)
            soup=bs4.BeautifulSoup(res.text,'html')
            Elements=soup.find_all("p","result-info")
        except requests.exceptions.ConnectionError:
              print("Connection refused")  
       
        for link in Elements:
           
            #I extract year model brand from title
            
            string=link.find("a","result-title hdrlnk").text
            string=string.lower()
            try:
                y=re.search(r"(\d{4})", string).group(1)
                y=float(y)
                year.append(y)
                
            except AttributeError:
                year.append(" ")
                string=link.find("a","result-title hdrlnk").text
                string=string.lower()
        
            titles.append(string)
            tile_links.append(link.find("a","result-title hdrlnk")['href'])
            p=k
            
            c0=0
            c1=0
            for category in p["make"]:
                if string.find(category)!=-1:
                    l=category
                    c1=c1+1
            if c1>0:
                make.append(l)
                r=p[p["make"]==l]
                for category in r["model"]:
                    if string.find(category)!=-1:
                        a=category
                        c0=c0+1
                if c0>0:
                    if a!="2" and a!="5" and a!="6" and a!="3":
                        model.append(a)
                    else:
                        model.append(" ")  
                else:
                    model.append(" ")
            else:
                make.append(" ")
                for category in p["model"]:
                    if string.find(category)!=-1:
                        a=category
                        c0=c0+1
                if c0>0:
                    if a!="2" and a!="5" and a!="6" and a!="3":
                        model.append(a)
                    else:
                        model.append(" ")  
                else:
                    model.append(" ")
           
            #Date stamp column
            now=dt.datetime.today().strftime("%m/%d/%Y")
            date.append(now)
            
            #Retrieves price
            p=link.find("span","result-meta")
            p=p.text
            p=p.split()
            p=p[0]
            p=p.replace(",","")
            p=p.replace("$","")
            try:
                p=float(p)
                prices.append(p)
            except ValueError:
                prices.append(None)
            
            #Retrieves and appends hood
            h=link.find('span',"result-hood")
            if (link.find("span","result-hood") != None):
                h=h.text
                h=h.replace("(","")
                h=h.replace(")","")
                hood.append(h.lower())
            else:
                hood.append(None)
        try:
            link1="https://phoenix.craigslist.org"+soup.find("a","button next")['href']
        except TypeError:
            " "
    dat={ 'Make':make,
         'Model':model,
         'Year':year,
         'Name':titles,
         "Links":tile_links,
         'Prices':prices,
         'Location':hood,
         'Last date':date}
    df = pd.DataFrame(dat)

    return(df)

In [70]:
df=craikilla(2000,3000,0,3,"cars")

805
https://phoenix.craigslist.org/search/sss?query=cars&sort=rel&min_price=2000&max_price=3000
7


In [72]:
df['Links'] = df['Links'].apply(lambda x: '<a href="{}">link</a>'.format(x))

df=df.to_html(escape=False)

In [73]:
df

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Make</th>\n      <th>Model</th>\n      <th>Year</th>\n      <th>Name</th>\n      <th>Links</th>\n      <th>Prices</th>\n      <th>Location</th>\n      <th>Last date</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>gmc</td>\n      <td></td>\n      <td></td>\n      <td>2002 gmc envoy four-door 4 x 4</td>\n      <td><a href="https://phoenix.craigslist.org/evl/cto/d/chandler-2002-gmc-envoy-four-door-4-4/7137192652.html">link</a></td>\n      <td>2900.0</td>\n      <td>scottsdale</td>\n      <td>06/06/2020</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>toyota</td>\n      <td>camry</td>\n      <td></td>\n      <td>2007 toyota camry</td>\n      <td><a href="https://phoenix.craigslist.org/nph/cto/d/phoenix-2007-toyota-camry/7137164080.html">link</a></td>\n      <td>3000.0</td>\n      <td>north phoenix/anthem</td>\n      <td>06/06/2020</td>\n    <